<a href="https://colab.research.google.com/github/alerosenberg/rais_ale/blob/main/Importar_RAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando bibliotecas


In [ ]:
!pip install wget 
!pip install py7zr
!pip install dask
!pip install dask[dataframe]
#!pip install sys
!pip install tqdm
#!pip install os
!pip install pandas
!pip install numpy
!pip install scipy
!pip install matplotlib
!pip install seaborn 
!pip install deflatebr

Importando bibliotecas

In [3]:
import wget
import py7zr
import dask 
import dask.dataframe as dd
import sys
from tqdm import tqdm
import py7zr
import os
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from statsmodels.graphics.factorplots import interaction_plot
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import matplotlib.pyplot as plt
import seaborn as sns
from deflatebr import deflate

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Transferir bases da RAIS para o Google Drive

In [4]:
# Acessar o site abaixo:
# ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/
# Escolher os anos e os estados desejados
# Descompatar o arquivo no computador
# Transferir o txt para o Google Drive
# Cada arquivo (utilizo como referência a RAIS de cada ano para o estado de SP) ocupa cerca de 6GB

Importar arquivos do Google Drive para o Google Colab

In [5]:
#STEP-1: Import libraries
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [7]:
#STEP-2: Autheticate E-Mail ID
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
# STEP-3: Get File from Drive using file-ID
# Após 'id': na linha abaixo é necessário trocar pelo caminho do arquivo desejado 
# O caminho pode ser obtido clicando com o botão direito sobre o arquivo e em Gerar Link
# Abaixo devem ser indicados os arquivos desejados (que já devem estar presentes no Drive)

downloaded08 = drive.CreateFile({'id':'1GGRz7aPkchIv6CWPzp0qM9IPMMFadJeV'})
downloaded08.GetContentFile('Estb2008.csv') 

Importando dados

In [58]:
#STEP-4: Read File
#3.1 Read file as panda dataframe
base08 = pd.read_csv('Estb2008.csv', sep=';', encoding="latin1", low_memory=False,
                        dtype={ 'Bairros SP': 'object',
                                'Distritos SP': 'object',
                                'Município': 'object',
                                'Natureza Jurídica': 'object',
                                'CNAE 2.0 Subclasse': 'object',
                                'CNAE 2.0 Classe': 'object',
                                'CNAE 95 Classe': 'object'})                 

Filtrar municípios do estado de São Paulo

In [88]:
sp = ['350010', 	'350020', 	'350030', 	'350040', 	'350050', 	'350055', 	'350060', 	'350070', 	'350075', 	'350080', 	'350090', 	'350100', 	'350110', 	'350115', 	'350120', 	'350130', 	'350140', 	'350150', 	'350160', 	'350170', 	'350180', 	'350190', 	'350200', 	'350210', 	'350220', 	'350230', 	'350240', 	'350250', 	'350260', 	'350270', 	'350275', 	'350280', 	'350290', 	'350300', 	'350310', 	'350315', 	'350320', 	'350330', 	'350335', 	'350340', 	'350350', 	'350360', 	'350370', 	'350380', 	'350390', 	'350395', 	'350400', 	'350410', 	'350420', 	'350430', 
'350440', 	'350450', 	'350460', 	'350470', 	'350480', 	'350490', 	'350500', 	'350510', 	'350520', 	'350530', 	'350535', 	'350540', 	'350550', 	'350560', 	'350570', 	'350580', 	'350590', 	'350600', 	'350610', 	'350620', 	'350630', 	'350635', 	'350640', 	'350650', 	'350660', 	'350670', 	'350680', 	'350690', 	'350700', 	'350710', 	'350715', 	'350720', 	'350730', 	'350740', 	'350745', 	'350750', 	'350760', 	'350770', 	'350775', 	'350780', 	'350790', 	'350800', 	'350810', 	'350820', 	'350830', 	'350840', 	'350850', 	'350860', 	'350870', 	'350880', 
'350890', 	'350900', 	'350910', 	'350920', 	'350925', 	'350930', 	'350940', 	'350945', 	'350950', 	'350960', 	'350970', 	'350980', 	'350990', 	'350995', 	'351000', 	'351010', 	'351015', 	'351020', 	'351030', 	'351040', 	'351050', 	'351060', 	'351070', 	'351080', 	'351090', 	'351100', 	'351110', 	'351120', 	'351130', 	'351140', 	'351150', 	'351160', 	'351170', 	'351190', 	'351200', 	'351210', 	'351220', 	'351230', 	'351240', 	'351250', 	'351260', 	'351270', 	'351280', 	'351290', 	'351300', 	'351310', 	'351320', 	'351330', 	'351340', 	'351350', 
'351360', 	'351370', 	'351380', 	'351385', 	'351390', 	'351400', 	'351410', 	'351420', 	'351430', 	'351440', 	'351450', 	'351460', 	'351470', 	'351480', 	'351490', 	'351492', 	'351495', 	'351500', 	'351510', 	'351512', 	'351515', 	'351518', 	'351519', 	'351520', 	'351530', 	'351535', 	'351540', 	'351550', 	'351560', 	'351565', 	'351570', 	'351580', 	'351590', 	'351600', 	'351610', 	'351620', 	'351630', 	'351640', 	'351650', 	'351660', 	'351670', 	'351680', 	'351685', 	'351690', 	'351700', 	'351710', 	'351720', 	'351730', 	'351740', 	'351750', 
'351760', 	'351770', 	'351780', 	'351790', 	'351800', 	'351810', 	'351820', 	'351830', 	'351840', 	'351850', 	'351860', 	'351870', 	'351880', 	'351885', 	'351890', 	'351900', 	'351905', 	'351907', 	'351910', 	'351920', 	'351925', 	'351930', 	'351940', 	'351950', 	'351960', 	'351970', 	'351980', 	'351990', 	'352000', 	'352010', 	'352020', 	'352030', 	'352040', 	'352042', 	'352044', 	'352050', 	'352060', 	'352070', 	'352080', 	'352090', 	'352100', 	'352110', 	'352115', 	'352120', 	'352130', 	'352140', 	'352150', 	'352160', 	'352170', 	'352180', 
'352190', 	'352200', 	'352210', 	'352215', 	'352220', 	'352230', 	'352240', 	'352250', 	'352260', 	'352265', 	'352270', 	'352280', 	'352290', 	'352300', 	'352310', 	'352320', 	'352330', 	'352340', 	'352350', 	'352360', 	'352370', 	'352380', 	'352390', 	'352400', 	'352410', 	'352420', 	'352430', 	'352440', 	'352450', 	'352460', 	'352470', 	'352480', 	'352490', 	'352500', 	'352510', 	'352520', 	'352530', 	'352540', 	'352550', 	'352560', 	'352570', 	'352580', 	'352585', 	'352590', 	'352600', 	'352610', 	'352620', 	'352630', 	'352640', 	'352650', 
'352660', 	'352670', 	'352680', 	'352690', 	'352700', 	'352710', 	'352720', 	'352725', 	'352730', 	'352740', 	'352750', 	'352760', 	'352770', 	'352780', 	'352790', 	'352800', 	'352810', 	'352820', 	'352830', 	'352840', 	'352850', 	'352860', 	'352870', 	'352880', 	'352885', 	'352890', 	'352900', 	'352910', 	'352920', 	'352930', 	'352940', 	'352950', 	'352960', 	'352965', 	'352970', 	'352980', 	'352990', 	'353000', 	'353010', 	'353020', 	'353030', 	'353040', 	'353050', 	'353060', 	'353070', 	'353080', 	'353090', 	'353100', 	'353110', 	'353120', 
'353130', 	'353140', 	'353150', 	'353160', 	'353170', 	'353180', 	'353190', 	'353200', 	'353205', 	'353210', 	'353215', 	'353220', 	'353230', 	'353240', 	'353250', 	'353260', 	'353270', 	'353280', 	'353282', 	'353284', 	'353286', 	'353290', 	'353300', 	'353310', 	'353320', 	'353325', 	'353330', 	'353340', 	'353350', 	'353360', 	'353370', 	'353380', 	'353390', 	'353400', 	'353410', 	'353420', 	'353430', 	'353440', 	'353450', 	'353460', 	'353470', 	'353475', 	'353480', 	'353490', 	'353500', 	'353510', 	'353520', 	'353530', 	'353540', 	'353550', 
'353560', 	'353570', 	'353580', 	'353590', 	'353600', 	'353610', 	'353620', 	'353625', 	'353630', 	'353640', 	'353650', 	'353657', 	'353660', 	'353670', 	'353680', 	'353690', 	'353700', 	'353710', 	'353715', 	'353720', 	'353730', 	'353740', 	'353750', 	'353760', 	'353770', 	'353780', 	'353790', 	'353800', 	'353810', 	'353820', 	'353830', 	'353850', 	'353860', 	'353870', 	'353880', 	'353890', 	'353900', 	'353910', 	'353920', 	'353930', 	'353940', 	'353950', 	'353960', 	'353970', 	'353980', 	'353990', 	'354000', 	'354010', 	'354020', 	'354025', 
'354030', 	'354040', 	'354050', 	'354060', 	'354070', 	'354075', 	'354080', 	'354085', 	'354090', 	'354100', 	'354105', 	'354110', 	'354120', 	'354130', 	'354140', 	'354150', 	'354160', 	'354165', 	'354170', 	'354180', 	'354190', 	'354200', 	'354210', 	'354220', 	'354230', 	'354240', 	'354250', 	'354260', 	'354270', 	'354280', 	'354290', 	'354300', 	'354310', 	'354320', 	'354323', 	'354325', 	'354330', 	'354340', 	'354350', 	'354360', 	'354370', 	'354380', 	'354390', 	'354400', 	'354410', 	'354420', 	'354425', 	'354430', 	'354440', 	'354450', 
'354460', 	'354470', 	'354480', 	'354490', 	'354500', 	'354510', 	'354515', 	'354520', 	'354530', 	'354540', 	'354550', 	'354560', 	'354570', 	'354580', 	'354600', 	'354610', 	'354620', 	'354625', 	'354630', 	'354640', 	'354650', 	'354660', 	'354670', 	'354680', 	'354690', 	'354700', 	'354710', 	'354720', 	'354730', 	'354740', 	'354750', 	'354760', 	'354765', 	'354770', 	'354780', 	'354790', 	'354800', 	'354805', 	'354810', 	'354820', 	'354830', 	'354840', 	'354850', 	'354860', 	'354870', 	'354880', 	'354890', 	'354900', 	'354910', 	'354920', 
'354925', 	'354930', 	'354940', 	'354950', 	'354960', 	'354970', 	'354980', 	'354990', 	'354995', 	'355000', 	'355010', 	'355020', 	'355030', 	'355040', 	'355050', 	'355060', 	'355070', 	'355080', 	'355090', 	'355100', 	'355110', 	'355120', 	'355130', 	'355140', 	'355150', 	'355160', 	'355170', 	'355180', 	'355190', 	'355200', 	'355210', 	'355220', 	'355230', 	'355240', 	'355250', 	'355255', 	'355260', 	'355270', 	'355280', 	'355290', 	'355300', 	'355310', 	'355320', 	'355330', 	'355340', 	'355350', 	'355360', 	'355365', 	'355370', 	'355380', 
'355385', 	'355390', 	'355395', 	'355400', 	'355410', 	'355420', 	'355430', 	'355440', 	'355450', 	'355460', 	'355465', 	'355470', 	'355475', 	'355480', 	'355490', 	'355495', 	'355500', 	'355510', 	'355520', 	'355530', 	'355535', 	'355540', 	'355550', 	'355560', 	'355570', 	'355580', 	'355590', 	'355600', 	'355610', 	'355620', 	'355630', 	'355635', 	'355640', 	'355645', 	'355650', 	'355660', 	'355670', 	'355680', 	'355690', 	'355695', 	'355700', 	'355710', 	'355715', 	'355720', 	'355730', 	'359999']
base08 = base08[base08.Município.isin(sp)]

Visualizar dados

In [89]:
base08.columns


Index(['Bairros_SP', 'CNAE_2_Classe', 'CNAE_95_Classe', 'Distritos_SP',
       'Qtd_Vínculos_CLT', 'Qtd_Vínculos_Ativos', 'Qtd_Vínculos_Estatutários',
       'Ind_Atividade_Ano', 'Ind_CEI_Vinculado', 'Ind_Estab_Participa_PAT',
       'Ind_Rais_Negativa', 'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'CNAE_2_Subclasse', 'Tamanho_Estabelecimento', 'Tipo_Estab',
       'Tipo_Estab1'],
      dtype='object')

Renomear colunas

In [61]:
base08.rename(columns = {'Bairros SP': 'Bairros_SP', 'CNAE 2.0 Classe': 'CNAE_2_Classe',
'CNAE 95 Classe': 'CNAE_95_Classe', 'Distritos SP': 'Distritos_SP',
'Qtd Vínculos CLT': 'Qtd_Vínculos_CLT', 'Qtd Vínculos Ativos': 'Qtd_Vínculos_Ativos', 
'Qtd Vínculos Estatutários': 'Qtd_Vínculos_Estatutários', 'Ind Atividade Ano': 'Ind_Atividade_Ano',
'Ind CEI Vinculado': 'Ind_CEI_Vinculado', 'Ind Estab Participa PAT': 'Ind_Estab_Participa_PAT',
'Ind Rais Negativa': 'Ind_Rais_Negativa', 'Ind Simples': 'Ind_Simples',
'Natureza Jurídica': 'Natureza_Jurídica', 'CNAE 2.0 Subclasse': 'CNAE_2_Subclasse',
'Tamanho Estabelecimento': 'Tamanho_Estabelecimento', 'Tipo Estab': 'Tipo_Estab',
'Tipo Estab.1': 'Tipo_Estab1','Bairros Fortaleza': 'Bairros_Fortaleza',
'Bairros RJ': 'Bairros_RJ', 'Regiões Adm DF': 'Regiões_Adm_DF'}, inplace=True)

In [62]:
base08.columns

Index(['Bairros_SP', 'Bairros_Fortaleza', 'Bairros_RJ', 'CNAE_2_Classe',
       'CNAE_95_Classe', 'Distritos_SP', 'Qtd_Vínculos_CLT',
       'Qtd_Vínculos_Ativos', 'Qtd_Vínculos_Estatutários', 'Ind_Atividade_Ano',
       'Ind_CEI_Vinculado', 'Ind_Estab_Participa_PAT', 'Ind_Rais_Negativa',
       'Ind_Simples', 'Município', 'Natureza_Jurídica', 'Regiões_Adm_DF',
       'CNAE_2_Subclasse', 'Tamanho_Estabelecimento', 'Tipo_Estab',
       'Tipo_Estab1'],
      dtype='object')

Seleção de colunas


In [63]:
base08 = base08.filter(items=['Bairros_SP','CNAE_2_Classe',
       'CNAE_95_Classe', 'Distritos_SP', 'Qtd_Vínculos_CLT',
       'Qtd_Vínculos_Ativos', 'Qtd_Vínculos_Estatutários', 'Ind_Atividade_Ano',
       'Ind_CEI_Vinculado', 'Ind_Estab_Participa_PAT', 'Ind_Rais_Negativa',
       'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'CNAE_2_Subclasse', 'Tamanho_Estabelecimento', 'Tipo_Estab','Tipo_Estab1'])
print(base08.shape)
base08.columns

(2132287, 18)


Index(['Bairros_SP', 'CNAE_2_Classe', 'CNAE_95_Classe', 'Distritos_SP',
       'Qtd_Vínculos_CLT', 'Qtd_Vínculos_Ativos', 'Qtd_Vínculos_Estatutários',
       'Ind_Atividade_Ano', 'Ind_CEI_Vinculado', 'Ind_Estab_Participa_PAT',
       'Ind_Rais_Negativa', 'Ind_Simples', 'Município', 'Natureza_Jurídica',
       'CNAE_2_Subclasse', 'Tamanho_Estabelecimento', 'Tipo_Estab',
       'Tipo_Estab1'],
      dtype='object')

In [64]:
base08[["Município"]].head(10)

,Município
2821015,354850
2821016,355030
2821017,352530
2821018,354340
2821019,350550
2821020,350600
2821021,351110
2821022,350950
2821023,351620
2821024,350610


In [65]:
base08.iloc[50000:50005]

,Bairros_SP,CNAE_2_Classe,CNAE_95_Classe,Distritos_SP,Qtd_Vínculos_CLT,Qtd_Vínculos_Ativos,Qtd_Vínculos_Estatutários,Ind_Atividade_Ano,Ind_CEI_Vinculado,Ind_Estab_Participa_PAT,Ind_Rais_Negativa,Ind_Simples,Município,Natureza_Jurídica,CNAE_2_Subclasse,Tamanho_Estabelecimento,Tipo_Estab,Tipo_Estab1
2871015,9999,49299,60259,9999,1,1,0,1,0,0,0,1,350370,2062,4929901,2,1,Cnpj
2871016,0021,14126,18120,0009,12,12,0,1,0,0,0,0,355030,2062,1412601,4,1,Cnpj
2871017,9999,56112,55220,9999,47,47,0,1,0,1,0,0,350950,2062,5611203,5,1,Cnpj
2871018,9999,56112,55220,9999,95,95,0,1,0,1,0,0,350950,2062,5611203,6,1,Cnpj
2871019,9999,18130,22225,9999,6,6,0,1,0,0,0,1,352690,2062,1813001,3,1,Cnpj


In [66]:
base08.sort_values("Município", ascending=True).head(5)

,Bairros_SP,CNAE_2_Classe,CNAE_95_Classe,Distritos_SP,Qtd_Vínculos_CLT,Qtd_Vínculos_Ativos,Qtd_Vínculos_Estatutários,Ind_Atividade_Ano,Ind_CEI_Vinculado,Ind_Estab_Participa_PAT,Ind_Rais_Negativa,Ind_Simples,Município,Natureza_Jurídica,CNAE_2_Subclasse,Tamanho_Estabelecimento,Tipo_Estab,Tipo_Estab1
3692118,9999,14126,18120,9999,0,0,0,0,0,0,1,0,350010,2062,1412601,1,1,Cnpj
3328470,9999,47440,52442,9999,7,7,0,1,0,0,0,0,350010,2062,4744001,3,1,Cnpj
4697293,9999,47636,52493,9999,0,0,0,1,0,0,1,1,350010,2062,4763604,1,1,Cnpj
4312687,9999,01512,01414,9999,0,0,0,1,0,0,1,0,350010,4080,0151201,1,1,Cnpj
2839263,9999,45307,50300,9999,5,5,0,1,0,0,0,1,350010,2135,4530703,3,1,Cnpj


In [98]:
cubo = base08.groupby(["CNAE_95_Classe","Qtd_Vínculos_CLT","Ind_Atividade_Ano","Ind_CEI_Vinculado",
"Ind_Rais_Negativa","Ind_Simples","Município","Natureza_Jurídica","Tamanho_Estabelecimento"])["CNAE_95_Classe"].count().reset_index(name="count")
cubo
# cubo = base08.groupby(['CNAE_95_Classe','Qtd_Vínculos_CLT','Ind_Atividade_Ano','Ind_CEI_Vinculado',
# 'Ind_Rais_Negativa','Ind_Simples','Município','Natureza_Jurídica','Tamanho_Estabelecimento']).count()

,CNAE_95_Classe,Qtd_Vínculos_CLT,Ind_Atividade_Ano,Ind_CEI_Vinculado,Ind_Rais_Negativa,Ind_Simples,Município,Natureza_Jurídica,Tamanho_Estabelecimento,count
0,000-1,0,1,1,0,0,352130,2062,1,1
1,000-1,1,1,1,0,0,351740,2062,2,1
2,000-1,3,1,1,0,0,352240,2062,2,1
3,01112,0,0,0,0,0,351740,3069,1,1
4,01112,0,0,0,0,0,352240,4080,1,2
...,...,...,...,...,...,...,...,...,...,...
573209,99007,186,1,0,0,0,353710,2054,7,1
573210,99007,200,1,0,0,0,354520,2062,7,1
573211,99007,208,1,0,0,0,354520,2062,7,1
573212,99007,219,1,0,0,0,354520,2062,7,1


Exportar como csv para o Drive

In [99]:
cubo.to_csv('cubo_rais_08.csv')
!cp cubo_rais_08.csv "drive/My Drive/Cubo"

In [ ]:

# # Corrige grau de escolaridade para obter valor 0
#     base['Escolaridade após 2005'] = pd.to_numeric(dados['Escolaridade após 2005'], errors="coerce")
#     dados['Escolaridade após 2005'] -= 1
#     # Coloca label em sexo
#     dados['Sexo Trabalhador'] = pd.to_numeric(dados['Sexo Trabalhador'],errors="coerce")
#     dados.loc[dados['Sexo Trabalhador'] == 1, 'Sexo Trabalhador'] = "Masculino"
#     dados.loc[dados['Sexo Trabalhador'] == 2, 'Sexo Trabalhador'] = "Feminino"
#     # Filtra apenas quem Raça Cor não é 99
#     dados['Raça Cor'] = pd.to_numeric(dados['Raça Cor'], errors="coerce")
#     dados = dados.loc[dados["Raça Cor"] != 99,:]
#     # Coloca label em raça
#     dados.loc[dados["Raça Cor"] == 1, 'Raça Cor'] = "Indígena"
#     dados.loc[dados["Raça Cor"] == 2, 'Raça Cor'] = "Branca"
#     dados.loc[dados["Raça Cor"] == 4, 'Raça Cor'] = "Preta"
#     dados.loc[dados["Raça Cor"] == 6, 'Raça Cor'] = "Amarela"
#     dados.loc[dados["Raça Cor"] == 8, 'Raça Cor'] = "Parda"
#     dados.loc[dados["Raça Cor"] == 9, 'Raça Cor'] = "Não identificada"
#     # Filtra apenas quem tem salário maior que 0 e idade de 18 a 80 anos
#     dados['Idade'] = pd.to_numeric(dados['Idade'], errors="coerce")
#     dados = dados.loc[(dados['Vl Remun Média Nom'] > 0) & 
#                         (dados['Idade'] > 17) & 
#                         (dados['Idade'] < 81), :]

#     # Cria variável binária Brasileiro
#     dados['Nacionalidade'] = pd.to_numeric(dados['Nacionalidade'], errors="coerce")
#     dados['Brasileiro'] = dados['Nacionalidade'].apply(lambda x: 1 if (x == 10) else 0)


In [ ]:
#  import tensorflow as tf
#  tf.test.gpu_device_name()
#!cat /proc/meminfo